In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from datasets.rgbd import Parser, Dataset

data_dir = '/run/media/junzhe/SLAM_DATA/data/museum/kitchen_rgbd_pcd/'
data_factor = 1

parser = Parser(
    data_dir=data_dir,
    factor=data_factor,
    normalize=True,
    test_every=10,
)


dataset = Dataset(parser, split="train", load_depths=True)
np.arange(len(parser.image_names))

In [ ]:
!python rgbd_trainer.py \
    --data-dir /run/media/junzhe/SLAM_DATA/data/museum/kitchen_rgbd_pcd/ \
    --dataset rgbd \
    --data-factor 1 \
    --result-dir /run/media/junzhe/SLAM_DATA/data/museum/gsplat/kitchen_rgbd/ \
    --depth-loss

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from datasets.slam_dataset import Parser, Dataset

data_dir = '/run/media/junzhe/SLAM_DATA/data/tartanair/scenes/abandonedfactory/Easy/P001/'
dataset_type = 'tartanair'
data_factor = 1

parser = Parser(
    data_dir=data_dir,
    dataset_type=dataset_type,
    factor=data_factor,
    normalize=True,
    test_every=10,
)


dataset = Dataset(parser, split="train", load_depths=True)

In [ ]:
!python rgbd_trainer.py \
    --data-dir /run/media/junzhe/SLAM_DATA/data/tartanair/scenes/hospital/Easy/P001/ \
    --dataset tartanair \
    --data-factor 1 \
    --result-dir /run/media/junzhe/SLAM_DATA/data/museum/gsplat/hospital_with_depth/ \
    --depth-loss

In [ ]:
!python rgbd_trainer.py --help

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from datasets.slam_dataset import Parser, Dataset
from spatialmath.base import tr2eul

data_dir = '/run/media/junzhe/SLAM_DATA/data/tum/rgbd_dataset_freiburg1_room/'
dataset_type = 'tum'
data_factor = 1

parser = Parser(
    data_dir=data_dir,
    dataset_type=dataset_type,
    factor=data_factor,
    normalize=True,
    test_every=10,
)


dataset = Dataset(parser, split="train", load_depths=True)

In [ ]:
!python rgbd_trainer.py \
    --data-dir /run/media/junzhe/SLAM_DATA/data/tum/rgbd_dataset_freiburg2_xyz/ \
    --dataset tum \
    --data-factor 1 \
    --result-dir /run/media/junzhe/SLAM_DATA/data/museum/gsplat/kitchen_rgbd/

In [ ]:
! python rgbd_trainer.py --help

In [ ]:
%reload_ext autoreload
%autoreload 2

from datasets.dataloader import TUMLoader, TartanAirLoader
import open3d as o3d
import numpy as np

data_dir = '/run/media/junzhe/SLAM_DATA/data/tartanair/scenes/hospital/Easy/P001/'
slam_dataset = TartanAirLoader(data_dir)

# data_dir = '/run/media/junzhe/SLAM_DATA/data/tum/rgbd_dataset_freiburg2_xyz/'
# slam_dataset = TUMLoader(data_dir)

slam_dataset.load_ground_truth()
stacked_pc = o3d.geometry.PointCloud()
fx, fy, cx, cy = slam_dataset.camera
image_width, image_height = slam_dataset.image_size
intrinsic = o3d.camera.PinholeCameraIntrinsic(image_width, image_height, fx, fy, cx, cy)
N = 30


# conv_T = np.array([
#     [0, -1, 0, 0],
#     [0, 0, 1, 0],
#     [1, 0, 0, 0],
#     [0, 0, 0, 1]
# ])
conv_T = np.array([
    [0,0,1,0],
    [0,1,0,0],
    [-1,0,0,0],
    [0,0,0,1]
])

stacked_pc = o3d.geometry.PointCloud()
for i in range(50,150,25):
    slam_dataset.set_curr_index(i)
    T = slam_dataset.read_current_ground_truth()
    print(T.t)
    rgb, depth = slam_dataset.read_current_rgbd()
    rgb = o3d.geometry.Image(rgb)
    depth = o3d.geometry.Image(depth.astype(np.float32))
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb, depth, convert_rgb_to_intensity=False, depth_trunc=100, depth_scale=1)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    # pcd.transform(conv_T)
    pcd.transform(T)
    stacked_pc += pcd


expected_number = 1e5
voxel_size = 0.01
while len(stacked_pc.points) > expected_number:
    voxel_size *= 1.1
    stacked_pc = stacked_pc.voxel_down_sample(voxel_size=voxel_size)
print(voxel_size, len(stacked_pc.points))
# o3d.visualization.draw_geometries([stacked_pc])

In [ ]:
traj = []
for i in range(0,slam_dataset.get_total_number()):
    slam_dataset.set_curr_index(i)
    T = slam_dataset.read_current_ground_truth()
    traj.append(T.t)
# plot traj with plotly
import plotly.graph_objects as go
import numpy as np
traj = np.array(traj)

# show index when mouse hover
text = [f'index: {i}' for i in range(traj.shape[0])]
fig = go.Figure(data=[go.Scatter3d(x=traj[:,0], y=traj[:,1], z=traj[:,2], mode='markers', text=text,marker=dict(size=3, color=traj[:,2], colorscale='Viridis', opacity=0.8))])
# use same scale for each axis
fig.update_layout(scene_aspectmode='cube')
fig.show()

# x-forward, y-down, z-left

In [ ]:
# visualize stacked_pc in rgb and xyz with plotly
import plotly.graph_objects as go
fig = go.Figure()
rgb = np.asarray(stacked_pc.colors)
xyz = np.asarray(stacked_pc.points)
fig.add_trace(go.Scatter3d(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], mode='markers', marker=dict(size=2, color=rgb, opacity=1)))
fig.update_layout(width=1000, height=1000)
fig.show()

In [ ]:
import open3d as o3d
import numpy as np
from spatialmath.base import q2r

# RGBD dataset
traj_file = '/run/media/junzhe/SLAM_DATA/data/museum/kitchen_rgbd_pcd/traj.tum'

traj_raw = []
with open(traj_file) as f:
    for line in f:
        traj_raw.append([float(x) for x in line.strip().split()])

traj = []
for i in range(len(traj_raw)):
    traj.append(traj_raw[i][1:4])

# plot traj with plotly
import plotly.graph_objects as go
import numpy as np
traj = np.array(traj)

# show index when mouse hover
text = [f'index: {i}' for i in range(traj.shape[0])]
fig = go.Figure(data=[go.Scatter3d(x=traj[:,0], y=traj[:,1], z=traj[:,2], mode='markers', text=text,marker=dict(size=3, color=traj[:,2], colorscale='Viridis', opacity=0.8))])
# use same scale for each axis
fig.update_layout(scene_aspectmode='cube')
fig.show()

# x-forward, y-down, z-left